# Drzewa i lasy
W czasie tego ćwiczenia będziemy przewidywali ceny samochodu Toyota Corolla, z wykorzystaniem 3 metod:

- drzew (1964)
- losowego lasu (1995)
- Ekstremalnie losowego lasu (2006)

## Wczytanie i przygotowanie danych

In [4]:
import pandas as pd
from sklearn import tree
from sklearn import metrics
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.tree import DecisionTreeRegressor

# Wczytanie danych
toyota_df = pd.read_csv('ToyotaCorolla.csv')
toyota_df = toyota_df.rename(columns={'age_08_04': 'age', 'quarterly_tax': 'tax'})
predictors = ['age', 'km', 'fuel_type', 'hp', 'met_color', 'automatic', 'cc', 
              'doors', 'tax', 'weight']
outcome = 'price'
X = pd.get_dummies(toyota_df[predictors], drop_first=True)
Y = toyota_df[outcome]

# [ToDo] Podziel próbkę na testową i uczącą w stosunku 40/60
train_X, test_X, train_y, test_y = train_test_split(X, Y, test_size=0.4, random_state=42)


C:\Users\424423\AppData\Local\Temp\ipykernel_3616\3724516347.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# Metoda 1 - drzewo regresyjne
Wykorzystując więdze z poprzednich zajęć zbuduj drzewo regresyjne. 

# Algorytm znalezienia najlepszego rozwiazania:
1. Wykorzystując Grid Search oraz podany `param_grid` znajdź najlepszą kombinację parametrów.
2. Wyświetl najlepsze drzewo.
3. Popraw działanie algorytmu poprzez random search budując `param_grid1` gdzie każdy z parametrów jest listą: `max_depth` oraz `min_sample_split` - plus minus 2 ze skokiem 1, 'min_impurity_decrease' - plus minus 0.002 ze skokiem 0.001.

Porównaj Otrzymane drzewa pod kątem RMSE oraz struktury.


In [11]:
# Zbuduj drzweo 
from sklearn.ensemble import RandomForestRegressor
from sklearn import tree

forest = RandomForestRegressor(bootstrap= True, max_samples =0.95)

# Parametry optymalizacji
param_grid = {
    'max_depth': [5, 10, 15, 20, 25], 
    'min_impurity_decrease': [0, 0.001, 0.005, 0.01], 
    'min_samples_split': [10, 20, 30, 40, 50], 
}
# [Komentarz] Wykorzystujemy nowy paramater min_impurity_decrease

# [ToDo] Wykorzystać GridSearch to znalezienia najlepszego drzewa
from sklearn.model_selection import  GridSearchCV
tree_gs = GridSearchCV(forest, param_grid, n_jobs=-1, cv=3, verbose=0)
fc=tree_gs.fit(train_X,train_y)
print('Najlepsze parametry drzewa grid search',tree_gs.best_params_)
best_params = tree_gs.best_params_
regTree = RandomForestRegressor(best_params)

# [ToDo] Narysować najlepsze drzewo
#best_params = tree_gs.best_params_
#best_tree = RandomForestRegressor(best_params)
#fit_forest = best_tree.fit(train_X, train_y)
#tree.plot_tree(fit_forest, feature_names=X.columns, filled=True)

# [ToDO] Reprezentacja tekstowa drzewa
text_representation = tree.export_text(regTree, feature_names=train_X.columns)
print(text_representation)

# [ToDO] ile wynosi MPE, MAE i RMSE - wykorzystaj funkcję KPI z poprzednich zajęć - próbka ucząca i testowa

# [ToDo] Parametry drugiego etapu dopasowania

# [ToDo] Wykorzystać RandomSearch do poprawy dopasoawnia

# [ToDo] Narysować drzewo

# [ToDo] Wyznaczyć RMSE dla próbek uczącej i testowej



Najlepsze parametry drzewa grid search {'max_depth': 15, 'min_impurity_decrease': 0, 'min_samples_split': 10}


InvalidParameterError: The 'decision_tree' parameter of export_text must be an instance of 'sklearn.tree._classes.DecisionTreeClassifier' or an instance of 'sklearn.tree._classes.DecisionTreeRegressor'. Got RandomForestRegressor(n_estimators={'max_depth': 15, 'min_impurity_decrease': 0,
                                    'min_samples_split': 10}) instead.

# Metoda 2 - losowy las
Losowy las wykorzystuje dwie istotne koncepcje: **the wisdom of crowd** oraz **ensemble model**.
- *Mądrość tłumów* - Idea ta wyjaśnia, że średnia opinia grupy ludzi będzie bardziej precyzyjna (średnio) niż opinia pojedynczego członka tej grupy.
- *Model zespołowy* - Model zespołowy to (meta)model składający się z wielu podmodeli.

## Jak wykorzystać Wisdom of Crowd w praktyce?
Generalnie są 3 pomysły jak wykorzystać Wisom of Crowd:

*Uczyć różnymi danymi*
**Bootstrap** - Każde drzewo otrzyma losowy wybór z początkowego zestawu danych treningowych. Ten losowy wybór jest dokonywany z zastępowaniem. W praktyce oznacza to, że te same punkty danych mogą pojawić się wielokrotnie w zestawie treningowym używanym przez każde drzewo, a niektóre inne punkty danych mogą po prostu nie być użyte.
**Ograniczenie Liczby Próbek** - ograniczenie ilości danych przekazywanych do każdego drzewa. W ten sposób ograniczasz ich trening, ale sprawiasz, że każde drzewo jest trochę inne. Liczba próbek może być ustawiona w scikit-learn za pomocą parametru `max_samples`. Zwykle ustawia się współczynnik (na przykład `0.9`), tak aby zachować (losową) część początkowego zestawu treningowego.

*Ograniczyć liczbę atrybutów z których może powstać drzewo* 

Parametr `max_features` w losowym lesie określa maksymalną liczbę cech rozważanych przy podziale w każdym węźle drzewa. Ustawienie tego parametru na mniejszą wartość sprawia, że każde drzewo jest bardziej zróżnicowane i redukuje korelację między drzewami, co zazwyczaj zwiększa ogólną wydajność modelu.


## Parametry do losowego lasu
Uruchamiająć losowy las następujące parametry uznajemy za istotne.

1. **bootstrap (domyślnie: true)**  Ten parametr określa, czy las losowy będzie korzystał z próbkowania bootstrap. Pozostawić tą wartość na *true*.

2. **max_depth (domyślnie: None)**  Podobnie jak w przypadku zwykłego drzewa, jest to maksymalna głębokość każdego drzewa (tj. maksymalna liczba kolejnych pytań tak/nie). Ustawmy ją na 7.

3. **max_features (domyślnie: 'auto')**  Jest to maksymalna liczba cech, z których drzewo może wybierać podczas dzielenia węzła. Zbiór dostępnych cech do wyboru losowo zmienia się w każdym węźle. Im niższa wartość max_features, tym większe różnice między drzewami lasu (co prowadzi do lepszego lasu), ale tym bardziej ograniczone jest każde drzewo (co skutkuje niższą dokładnością poszczególnych drzew). 

4. **min_samples_leaf (domyślnie: 1)**  
   Podobnie jak w przypadku zwykłego drzewa, jest to minimalna liczba próbek, które musi zawierać każdy liść. Niska wartość pozwoli drzewu bardziej się rozwinąć, co najprawdopodobniej zwiększy dokładność na zbiorze treningowym, ale za cenę ryzyka przeuczenia.

5. **min_samples_split (domyślnie: 2)**  Podobnie jak w przypadku zwykłego drzewa, jest to minimalna liczba próbek, jakie musi zawierać węzeł, aby został podzielony. Podobnie jak w przypadku min_samples_leaf, niska wartość najprawdopodobniej zwiększy dokładność na zbiorze treningowym, ale może nie poprawić zbioru testowego, a nawet mu zaszkodzić.
6. **n_estimators (domyślnie: 100)**   liczba drzew w lesie. Im więcej, tym lepiej - kosztem dłuższego czasu obliczeń. W pewnym momencie dodatkowy czas działania nie będzie wart minimalnej poprawy.

8. **criterion (domyślnie: 'mse')**  Jest to wskaźnik, który algorytm będzie minimalizować. Wybierz 'mse', aby zoptymalizować MSE, lub 'mae' dla MAE. lgorytm tworzenia lasu losowego będzie potrzebował znacznie więcej czasu na optymalizację dla MAE niż dla MSE.

# Algorytm znalezienia najlepszego rozwiazania:
1. Wykorzystując Grid Search oraz podany `param_grid` znajdź najlepszą kombinację parametrów.
2. Wyświetl najlepsze drzewo.
3. Popraw działanie algorytmu poprzez random search budując `param_grid1` gdzie każdy z parametrów jest listą: `max_depth` oraz `min_sample_split` - plus minus 2 ze skokiem 1, 'min_impurity_decrease' - plus minus 0.002 ze skokiem 0.001.

Porównaj Otrzymane drzewa pod kątem RMSE oraz struktury.



In [ ]:
from sklearn.ensemble import RandomForestRegressor
# Przykład działania estymatora - tego kodu nie uruchamiamy ;)
forest = RandomForestRegressor(n_jobs=1, n_estimators=30)

# forest.fit(X_train,Y_train)

# Parametry do optymalizacji
max_depth = list(range(5,11)) + [None]
min_samples_split = range(5,20)
min_samples_leaf = range(2,15)
max_features = range(3,8)
bootstrap = [True] 
max_samples = [.7,.8,.9,.95,1]

param_dist = {'max_depth': max_depth,
              'min_samples_split': min_samples_split,
              'min_samples_leaf': min_samples_leaf,
              'max_features': max_features,
              'bootstrap': bootstrap,
              'max_samples': max_samples}

# [ToDo] Dokonaj optymalizacji parametrów z wykorzystaniem random Search. Parametry do Random Search
# cv = 6, verbose = 2, n_iter = 400, scoring='neg_mean_absolute_error'
# forest = RandomForestRegressor(n_jobs=1, n_estimators=30)

# [ToDo] Wyznacz błąd dla najlepszego modelu - czy zwiększenie n_estimators do 200 poprawia wynik


# Interpretacja modelu
Do interpretacji modelu wykorzystujemy feature_importance (znaczeni cech). W bibliotece sci_learn znaczenie cechy to redukcja, którą wnosi do celu, który algorytm stara się zminimalizować. W naszym przypadku chcemy obniżyć MAE (średni błąd absolutny). Zatem znaczenie cechy mierzone jest jako dokładność prognozy, którą wnosi każda cecha. Znaczenie cech jest następnie normalizowane (tzn. znaczenie każdej cechy jest skalowane tak, aby suma wszystkich znaczeń cech wynosiła 1).


In [ ]:
# [Todo] Przerób kod aby uzyskać wykres ważności cech
#cols = X_train.shape[1]  # number of columns in our training sets
# features = [f'M-{cols-col}' for col in range(cols)]
#data = forest.feature_importances_.reshape(-1,1)
#imp = pd.DataFrame(data=data, index=features, columns=['Forest'])
# imp.plot(kind='bar')


# Ekstremalnie losowy las
Pomysł na losowy las polegał na tym, że moglibyśmy uzyskać lepszą dokładność prognozy, biorąc średnią przewidywań wielu różnych drzew. W 2006 roku belgijscy badacze Pierre Geurts, Damien Ernst i Louis Wehenkel wprowadzili trzeci pomysł, aby jeszcze bardziej zwiększyć różnice między każdym drzewem. Na każdym węźle algorytm teraz losowo wybiera punkt podziału dla każdej cechy, a następnie wybiera najlepszy podział spośród nich. Choć to jak działa Extremely Randomized Trees (ETR) dziala, czyli losowo wybiera punkty podziału, wydaje się nieintuicyjne, to jednak zwiększa to jeszcze bardziej różnice między każdym drzewem w ETR, co skutkuje lepszą ogólną dokładnością. 

In [ ]:
#from sklearn.ensemble import ExtraTreesRegressor
#ETR = ExtraTreesRegressor(n_jobs=-1, n_estimators=200,
                          min_samples_split=15, min_samples_leaf=4, max_samples=0.95,
                          max_features=4, max_depth=8, bootstrap=True)
#ETR.fit(X_train,Y_train)

# Macież parametrów # Parametry do optymalizacji
max_depth = list(range(5,11)) + [None]
min_samples_split = range(5,20)
min_samples_leaf = range(2,15)
max_features = range(3,8)
bootstrap = [True] 
max_samples = [.7,.8,.9,.95,1]

param_dist = {'max_depth': max_depth,
              'min_samples_split': min_samples_split,
              'min_samples_leaf': min_samples_leaf,
              'max_features': max_features,
              'bootstrap': bootstrap,
              'max_samples': max_samples}

# [ToDo] Dokonaj optymalizacji parametrów z wykorzystaniem random Search. Parametry do Random Search
# cv = 6, verbose = 2, n_iter = 400, scoring='neg_mean_absolute_error'

